#####JAStark DAT8
#Namus: Unidentified Persons</h1>


##Using Machine Learning to Predict the “Identification Potential” 5-star rating

##1. Background 

Missing and unidentified persons referred to as the nation's silent mass disaster. There are many websites - from amateur to state-run - that attempt to facilitate finding and naming these individuals. One such site, funded by the Department of Justice, is the National Missing and Unidentified Persons System (NamUs) which does this at the nationwide level. It consists of three databases: Missing Persons, Unidentified Persons, and Unclaimed Persons. This project focusses on data from the [Unidentified Persons](https://identifyus.org/en) database.  

##2. Problem

<img src="images/NamUs.png", width="50%" height="50%"/>

The database rates each case with an Identification Potential. Although it appears to be a 5-star rating, it is in face 6 (`'Extremily Low', 'Low, Low-Medium', 'Medium', 'Medium-High', `and`  'High'`). My goal is to predict this rating based on the data that is freely available to the public within the database. 

**If I *can* predict the rating:**
* Which features predict “Identification Potential”? Could they be used to search for matches against the Missing Persons’ Database more efficiently/effectively?
* Depending on how investigations work, this could be used to focus time and attention on those predictive features.
* This could provide incentive for other organisations and databases to employ a similar strategy, and facilitate performance comparisons between databases..

**If I can*not* predict the rating:**
* This may indicate key information is not available to the public on NamUs
* The star-rating may be determined by more than one person. 
* If it *is* determined by several different people, then within and between person variability could lead to inconsistent ratings at best, and arbitrary ratings at worst.

**Modelling**

This is a regression problem since I am looking to predict a rating on a scale of 1-6.



##3. Data

Data were scraped using BeautifulSoup4 and Requests. The data were scraped from the site by looping though case numbers with `range(0, 14000)` using [Namus_scrape2.py](./Namus_scrape2.py). This script called [get_html.py](get_html.py) which submit the request using the case number from Namus_scrape2.py, and made it look nice with BeautfulSoup4, and saved it to an html file identifed by case.

Next, data was extracted and put into a dataframe using [Namus_fromFile.py](./Namus_fromFile.py) and [get_namus_fromFile.py](get_namus_fromFile.py). These scripts again created a range of case numbers to loop through, identify a case.html file, called get_namus_fromFile.py to collect the specific data from that html file, and then append it to a list. At the end, the list would be converted to a dataframe. Originally, the two steps were performed in one: loop through the case numbers, submit a request to the website, save to an html file, extract the data, append to a list, and create a dataframe. All data was extracted from the HTML as is, except for images which were converted to a count, rather than getting the actual images themselves. The idea being that more pictures might lead to a higher identification potential and the fact that I was not going to get into image analysis.

**Challenge:** The number of case files collected were fewer than the number of cases on the website. Not every number has a case (for example, if it has been solved, that case number will no longer be available). But I was missing more than just solved cases, which was made clear when I found cases on the website that I did *not* have on file. It is possible that the missing cases were due to a timeout issue. Whatever the cause, it was fixed when I called **get_html** and **get_namus_fromFile** separately. The loop was rewritten to avoid collecting html files that I already had, and only collected the files it had previously missed. These were stored in a separate directory. 

Once all the html files were collected, and all the data from them collected, a dataframe consisting of the first "batch" of data was created and pickled. The same was done for the second "missing data" batch. I kept them separate to begin with in case the reason they were missed initially was due to something about the data itself. Once unpickled (same process as reading in a csv) the two dataframes were joined together to achieve a complete dataset, which you can see in [this notebook](./namus_testing.ipynb). 

###3.1 Brief Overview of the Data

The data consists of short and long-form text, binary (yes = 1, no = 0), and continuous integers, cells containing a mixture of text and integers, and datetime. 

Here is a very brief overview of the data:

**Shape** `(10529, 72)`  -> 10529 cases, 72 features

**Case Status** 

* `Unidentified           10527`
* `Unidentified Living        2`
 
**Sex**

* `Male      7815`
* `Female    2194`
* `Unsure     518`
* `           2 `  -> missing values
 
**Race**

* `White                     5357`
* `Unsure                    2001`
* `Black/African American    1838`
* `Other                     1006`
* `Asian                      216`
* `Native American             89`
* `                            22 `   -> missing              

###3.2 Map coming soon!

I hope to create a map with all the 'find' locations. About half the cases have an address associated with them, but I think a map would provide some context to the data, and *may* provide additional insight. 

##4. Feature selection (and analysis?)

Because I have many features, and of different data types, I wanted to start with a balance of features that were informative while also easily accessible (features not needing much cleaning or recoding). 

* For lots of details on the Feature Selection/Engineering thus far, please refer to the [notebook](namus_testing.ipynb)

##5. Modelling

The cell below shows the features included in the first linear regression model. They are all binary originally (i.e. I did not recode these), with `'1'` coding for 'yes - present or true' and `'0'` coding for 'no - not present or not true'. This is *not* true for `'images'`, however, which is a count of how many images are provided with that case. 

For example, a `'1'` for `'all_parts_recovered'` means that **yes** all parts were recovered. If that was a `'0'` for **no - not all parts recovered**, then `'organ_absent'` might be a `'1'` for **yes - organs absent**. 

* Sadly for me, _testing accuracy_ for this model returns an error. I'll figure out what I did wrong...

* In the mean time, I _can_ run RMSE tests on the model, where I get a value of 1.17. Not great when you compare that with the Null RSME which is 1.19.

* **The best model so far, measured using RMSE, is below, with a value of 0.82, compared to the Null at 1.19:**
* **Recoded features include `'_face'`, `'_fingerprints'`, `'_dna'`, and `'_dental'`.**

In [ ]:
#  Create features for linear regression: all features consisting of an integer
linreg_features3 = ['amputations',
                   '_face',
                  'artificial_parts_aids',
                  'deformities',
                  'finger_toe_nails',
                  'foreign_objects',
                  'head_not_recovered',
                  'images',
                  'medical_implants',
                  'n-hands_not_recovered',
                  'n-limbs_not_recovered',
                  'organ_absent',
                  'other_distinctive_features',
                  'piercings',
                  'prior_surgery',
                  'scars_and_marks',
                  'skeletal_findings',
                  '_fingerprints',
                  '_dna',
                  '_dental']

Most of my time has been spent exploring the data and making things nice. I still have lots of "making nice" to do. I also think that Linear Regression is not ideal, and am interested to learn about different regression models next week! Also, of course, I want to add text analysis, which I need to think about some more, once I have number-features all figured out :D

I also intend to perform more rigorous/varied model testing, and not rely only on RMSE (obvi).

###5.1 Linear modelling using the binary and continuous variable data

###5.2 Some kind of text analysis using the short and long-form text data

###5.3 Combine these things??

##6. Conclusions and Future Work